# パッシブゴールクリエイターの実装
* 『LangChainとLangGraphによるRAG・AIエージェント実践入門（12章より)』
* ユーザーからの具体的な目標を抽出するパターン

In [1]:
# !pip install langchain-core==0.3.0 langchain-community==0.3.0 \
# langgraph==0.2.22 langchain-openai==0.2.0 langchain-anthropic==0.2.0 \
# numpy==1.26.4 faiss-cpu==1.8.0.post1 \
# pydantic-settings==2.5.2 retry==0.9.2 decorator==4.4.2

In [2]:
# !pip install "pydantic<2.11"  # 例: 2.10.6 エラー回避用

In [1]:
from dotenv import load_dotenv
load_dotenv()

In [2]:
# import os

# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

## 「目標」を表現するデータモデル

In [4]:
class Goal(BaseModel):
    description: str = Field(..., description="目標の説明")

    # textプロパティを設定することで、目標の値を文字列として取得できるようにする
    @property
    def text(self) -> str:
        return f"{self.description}"

## プロンプト
* ここでは、明確で実行可能な目標についてはLLM自身に考えてもらうプロンプト

In [5]:
class PassiveGoalCreator:
    def __init__(
        self,
        llm: ChatOpenAI,
    ):
        self.llm = llm

    def run(self, query: str) -> Goal:
        prompt = ChatPromptTemplate.from_template(
            "ユーザーの入力を分析し、明確で実行可能な目標を生成してください。\n"
            "要件:\n"
            "1. 目標は具体的かつ明確であり、実行可能なレベルで詳細化されている必要があります。\n"
            "2. あなたが実行可能な行動は以下の行動だけです。\n"
            "   - インターネットを利用して、目標を達成するための調査を行う。\n"
            "   - ユーザーのためのレポートを生成する。\n"
            "3. 決して2.以外の行動を取ってはいけません。\n"
            "ユーザーの入力: {query}"
        )
        chain = prompt | self.llm.with_structured_output(Goal)
        return chain.invoke({"query": query})

## 実行

In [6]:
query = "カレーライスの作り方"

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

goal_creator = PassiveGoalCreator(llm=llm)

result: Goal = goal_creator.run(query=query)

# `@property`により、textメソッで呼び出せるようになっている
print(f"{result.text}")

カレーライスの作り方を調査し、ユーザーにわかりやすく説明するレポートを生成する。


「カレーライスの作り方」というあいまいな要求が、「カレーライスの作り方を調査し、ユーザーにわかりやすく説明するレポートを生成する。」という具体的な目標に改善されている。

# @propertyデコレーターについて


Python の **`@property` デコレータ** は、クラス内のメソッドを “関数呼び出し” ではなく **属性アクセスのように使えるプロパティ** に変換する仕組みです。これにより「外部 API からはシンプルなフィールドに見えるが、内部では計算・検証・キャッシュなどを挟みたい」という要件をスマートに満たせます。提示コードでは `Goal.text` を定義することで、`Goal` インスタンスに対して `goal.text` と書くだけで説明付き文字列が得られる――そして `goal.text()` のように丸括弧を付ける必要がなくなる――というわけです。以下で詳しく仕組みとメリット、使い方を解説します。

---

## 1. `@property` の基本動作

### 1.1 属性アクセスに変換

`@property` を付けたメソッドは **呼び出し不要で値を返す属性** になります。([geeksforgeeks.org][1])

```python
class Point:
    def __init__(self, x):
        self._x = x          # 本体は“隠し”属性に保持

    @property
    def x(self):
        return self._x       # ここで検証・加工も可能
```

呼び出し側は `p = Point(3); print(p.x)` と **属性扱い**で取得でき、`p.x()` とは書きません。

### 1.2 バックエンド値のカプセル化

* 後でバリデーションを追加したくなっても **外部 API を壊さず** に getter／setter を入れ替えられる点が大きな利点です。([stackoverflow.com][2])
* さらに `@<prop>.setter`、`@<prop>.deleter` を追加すれば書き込み・削除時の処理も制御できます。([stackoverflow.com][3])

---

## 2. コード例で見る `Goal.text` プロパティ

```python
class Goal(BaseModel):
    description: str = Field(..., description="目標の説明")

    @property
    def text(self) -> str:
        return f"{self.description}"
```

1. **`description`** は pydantic フィールド。
2. **`text` プロパティ**は `description` をフォーマットして返すだけだが、呼び出し側は

   ```python
   goal = Goal(description="簿記3級の合格")
   print(goal.text)  # ⇒ "簿記3級の合格"
   ```

   とシンプルに使える。

### 2.1 pydantic との相性

* Pydantic v2 では **プロパティは通常フィールドとして検証対象にならず**、モデルのメタデータにも入らないため「派生値」や「表示用文字列」を定義する場所として適切です。([docs.pydantic.dev][4])

---

## 3. 典型的なユースケース

| ユースケース                      | 解説                                                     |
| --------------------------- | ------------------------------------------------------ |
| **読み取り専用の派生値**              | 距離、面積、文字列連結など計算結果をキャッシュせず返す。([stratascratch.com][5])   |
| **バリデーション付き getter／setter** | 値をセットする際に型の整合や範囲チェックを行う。([realpython.com][6])          |
| **遅延計算＋メモ化**                | 計算コストの高い値を初アクセス時に計算し、別変数に保持。                           |
| **後方互換 API**                | もともと公開属性だったものを内部実装変更しても外部インターフェイスを保つ。([reddit.com][7]) |

---

## 4. 使い方の注意点

1. **重い処理を毎回行わない**

   * 毎アクセスで計算するとパフォーマンスに影響。必要なら内部にキャッシュ変数を置く設計に。
2. **サブクラスで拡張しにくい**

   * `@property` はメソッド名を再定義しなければオーバーライドできないため、複雑な多重継承では注意。([stackoverflow.com][2])
3. **pydantic フィールドとは独立**

   * プロパティはスキーマに含まれないので JSON シリアライズには出てこない（必要なら `@model_serializer` を使う）。([stackoverflow.com][8])

---

## 5. まとめ

* **`@property` = メソッド → 読み取り専用属性**
  *利点*: シンプルなアクセス記法・カプセル化・後方互換。
* **提示コードの意図**
  `Goal.text` により `Goal` インスタンスをそのまま LLMS の `with_structured_output` で使ったり、`goal.text` をプロンプト挿入文字列として再利用したりできる。
* **pydantic モデルでの役割**
  スキーマに含まず派生値・表示ラベルを提供する安全な方法。

以上が `@property` の機能と、提示コードでの位置づけです。これによりクラス利用側は **「メソッド呼び出しの煩わしさなく派生値を受け取れる」** というメリットを享受できます。

[1]: https://www.geeksforgeeks.org/python/python-property-decorator-property/?utm_source=chatgpt.com "Python Property Decorator - GeeksforGeeks"
[2]: https://stackoverflow.com/questions/6618002/using-property-versus-getters-and-setters?utm_source=chatgpt.com "Using @property versus getters and setters - python - Stack Overflow"
[3]: https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python?utm_source=chatgpt.com "How does the @property decorator work in Python? - Stack Overflow"
[4]: https://docs.pydantic.dev/2.4/concepts/models/?utm_source=chatgpt.com "Models - Pydantic"
[5]: https://www.stratascratch.com/blog/how-to-use-python-property-decorator-with-examples/?utm_source=chatgpt.com "How to Use Python Property Decorator (With Examples)"
[6]: https://realpython.com/python-getter-setter/?utm_source=chatgpt.com "Getters and Setters: Manage Attributes in Python"
[7]: https://www.reddit.com/r/learnpython/comments/osqvme/what_is_the_property_decorator_meant_to_be_used/?utm_source=chatgpt.com "What is the @property decorator meant to be used for? : r/learnpython"
[8]: https://stackoverflow.com/questions/63264888/pydantic-using-property-getter-decorator-for-a-field-with-an-alias?utm_source=chatgpt.com "pydantic: Using property.getter decorator for a field with an alias"
